<a href="https://colab.research.google.com/github/Santhosh-1801/DL/blob/main/CNNOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.15.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
fashion_mnist

<module 'keras.api._v2.keras.datasets.fashion_mnist' from '/usr/local/lib/python3.10/dist-packages/keras/api/_v2/keras/datasets/fashion_mnist/__init__.py'>

In [6]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)


In [10]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [11]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters


In [12]:
tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")

In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [15]:
model=tuner_search.get_best_models(num_models=1)[0]

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        20784     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 96)                2654304   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 2676538 (10.21 MB)
Trainable params: 2676538 (10.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1204 - accuracy: 0.9560 - val_loss: 0.2612 - val_accuracy: 0.9152
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0835 - accuracy: 0.9695 - val_loss: 0.2988 - val_accuracy: 0.9163
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0562 - accuracy: 0.9793 - val_loss: 0.3375 - val_accuracy: 0.9143
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0392 - accuracy: 0.9860 - val_loss: 0.3837 - val_accuracy: 0.9137
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0307 - accuracy: 0.9888 - val_loss: 0.4458 - val_accuracy: 0.9147
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0241 - accuracy: 0.9917 - val_loss: 0.4838 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0198 - accuracy: 0.9926 - val_loss: 0.5376 - val_accuracy